# configurar el proyecto django en jupyter para usar sus modelos y demas funciones

In [ ]:
# Configuración de Django
import sys
import os
from IPython.display import display, Markdown

# Obtiene el directorio actual del notebook
notebook_dir = os.getcwd()

# Añade el directorio del proyecto al path
sys.path.append(os.path.dirname(os.path.join(notebook_dir, '..','..')))

# Establece la variable de entorno para la configuración de Django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'core.settings')

import django
django.setup()


## funciones para imprimir reportes

In [ ]:
from IPython.display import display, HTML

def mostrar_inicio_formulas_principales(str, description):
    display(HTML(f"""
    <hr>
            <h1>{str}</h2>
            <small>{description}</small>
            <br>

    """))
def mostrar_inicio_formula(str, descripcion=None):
    display(HTML(f"""
    <hr>
            <h2>{str}</h2>
            <small>{descripcion}</small>

"""))
def mostrar_inicio_subformula(str, descripcion=None):
    display(HTML(f"""
    <hr>
            <h3>{str}</h3>
            <small>{descripcion}</small>

"""))
def mostrar_semaforo(color):
    # Mapea el color de la luz a estilos de colores
    colores = {
        "rojo": {"rojo": "red", "amarillo": "gray", "verde": "gray"},
        "amarillo": {"rojo": "gray", "amarillo": "yellow", "verde": "gray"},
        "verde": {"rojo": "gray", "amarillo": "gray", "verde": "green"}
    }
    
    # Establece el color de cada luz
    luces = colores.get(color.lower(), {"rojo": "gray", "amarillo": "gray", "verde": "gray"})
    
    # Genera el HTML con el semáforo y el color de luz seleccionado
    display(HTML(f"""
    <div style="border: 2px solid #333; border-radius: 10px; width: 50px; padding: 10px; text-align: center; background: #444;">
        <div style="width: 30px; height: 30px; background-color: {luces['rojo']}; border-radius: 50%; margin: 5px auto;"></div>
        <div style="width: 30px; height: 30px; background-color: {luces['amarillo']}; border-radius: 50%; margin: 5px auto;"></div>
        <div style="width: 30px; height: 30px; background-color: {luces['verde']}; border-radius: 50%; margin: 5px auto;"></div>
    </div>
    """))
    
def separador():
    display(HTML("<hr>"))

def mostrar_en_tabla_dinamica(titulo, values_dic):
    # Extraer los encabezados y valores de los datos del diccionario
    headers = list(values_dic.keys())
    values = list(values_dic.values())
    
    # Generar la tabla dinámica con los encabezados y valores del diccionario
    display(HTML(f"""
    <h3>{titulo}</h3>
    <table border="1" cellpadding="5" cellspacing="0">
        <tr>
            {''.join(f"<th>{header}</th>" for header in headers)}
        </tr>
        <tr>
            {''.join(f"<td>{value}</td>" for value in values)}
        </tr>
    </table>
    """))

def mostrar_resultado(resultado ):
    display(HTML(f"""
    <fieldset border="1" cellpadding="5" cellspacing="0">
            <legend>Resultado</legend>
            <p>{str(resultado)}</p>
    </fieldset>
    
    """))

def mostrar_formula(str):
    display(HTML(f"""
    <h3>{str}</h3>
    """))


# clase encargada de administrar el semaforo

In [ ]:

from math import atan, sqrt
import math

import pandas as pd
from applications.fandesign.mixins import presion_total
from applications.getdata.models import SensorsData, VdfData
from modules.queries import get_10min_sensor_data, get_10min_vdf_data
from IPython.display import display, Markdown

class Semaforo:
    """
    Clase que representa un semaforo de control de estado de la ventilacion del sistema de ventilacion de la mina,
    el semaforo tiene  7 variables para medir, que pueden retornar verde, rojo, o amarillo cada una de sus funciones, a partir
    de la sumatoria de cada resultado de las 7 variables se pondera el estado final.
    """
    def __init__(self):
        self.estado = 'verde'
        self.sensorData = None
        self.vdfData = None
        self.project = None
        self.Q1 = None 
        self.Q2 = None 
    
    def encender(self, project):
        self.vdfData = get_10min_vdf_data()
        self.sensorData = get_10min_sensor_data()

        dataframe_transpose = self.sensorData.mean().round(3)

        dataframe_sensor = pd.DataFrame([dataframe_transpose])

        dict_sensor = dict(dataframe_sensor.iloc[0].to_dict())
        self.project = project
        mostrar_inicio_formulas_principales("Se ha iniciado los calculos del semaforo", f"se ha asignado el proyecto con el ventilador: {self.project.ventilador}")
        mostrar_en_tabla_dinamica("valores del sensor - promediado en 10 minutos", dict_sensor)

        # mostrar vdf 
        dataframe_vdfDatatranspose = self.vdfData.mean().round(3).to_frame().transpose() 
        dataframe_vdfDatatranspose = dataframe_vdfDatatranspose.iloc[0]
        dataframe_vdfDatatranspose = dataframe_vdfDatatranspose.iloc[1:]
        dict_vdf = dict(dataframe_vdfDatatranspose.to_dict())

        mostrar_en_tabla_dinamica("valores del VDF - promediado en 10 minutos", dict_vdf)

    def calculate_tbh(self, tbs, hr):
        # tbh1 =E13*ATAN(0.151977*  SQRT(E8+8.313659))+     ATAN(E13+E8)-   ATAN(E8-1.6763)+    0.00391838*     POWER(E8,1.5)*ATAN(0.023101*E8)-4.686
        #  = E16 *ATAN(0.151977* SQRT(E17+8.313659))+    ATAN(E16+E17)-  ATAN(E17-1.6763)+   0.00391838*     POWER(E17,1.5)*ATAN(0.023101*E17)-4.686
        return tbs * atan(0.151977 * sqrt(hr + 8.313659)) + atan(tbs + hr) - atan(hr - 1.6763) + 0.00391838 * pow(hr, 1.5) * atan(0.023101 * hr) - 4.686035

    def calcular_area_ducto(self):
        mostrar_inicio_subformula("Calculando el area del ducto")
        area_ducto = None
        if self.project.ducto.t_ducto == "circular":
            area_ducto = 3.14159 * self.project.ducto.diametro**2

        if self.project.ducto.t_ducto == "ovalado":
            area_ducto = self.project.ducto.area
        
        if area_ducto == None:
            raise ValueError("NO hemos calculado el area del ducto correctamente.")
        mostrar_resultado(area_ducto)
        return area_ducto

    def calcular_velocidad_sensor(self, tbs, hr, P, pt, ps, tipo):

        # set de operaciones
        Tbh2 = self.calculate_tbh(tbs, hr)
        # =616.6*   EXP(17.27*E13/(237.3+E13))

        esd =  616.6*math.exp((17.27 * tbs) / (237.3 + tbs)) # Calcula la presión de vapor del agua a temperatura seca
        # =616.6    *   EXP(17.27*J6/(237.3+J6))
        esw = 616.6*math.exp((17.27 * Tbh2) / (237.3 + Tbh2)) # Calcula la presión de vapor del agua a temperatura húmeda
        Xs = 0.622*esw/(P-esw) # humedad absoluta en kg vapor de agua por kg aire seco
        Lw = (2502.5-2.386*Tbh2)*1000 # J/kg 
        S = Lw*Xs + 1005*Tbh2; # J/kg
        X = (S-1005*tbs)/(Lw+1884*(tbs-Tbh2))
        e = P*X/(0.622+X); #  Pa 

        mostrar_inicio_formula("calculando la velocidad del sensor", tipo)

        # crear un diccionario de los valores, para imprimir en la tabla dinamica
        values_dic = {
            'tbh2': Tbh2,
            'esd': esd,
            'esw': esw,
            'Xs': Xs,
            'Lw': Lw,
            'S': S,
            'X': X,
            'e': e
        }

        titulo = "Variables de entrada"
        # mostrar_reporte(titulo, Tbh2, esd, esw, Xs, Lw, S, X, e)
        mostrar_en_tabla_dinamica(titulo, values_dic)
        densidad_aire_frente = (P-e)/(287.04*(tbs+273.15)); #  Kg aire seco/m3

        # =SQRT(2*(E5-E6)/E22)
        velocidad_sensor = sqrt(2*(pt-ps)/densidad_aire_frente) # Densidad aire en la frente)   2 decimales >>> velocidad aire sensor frente

        
        values_dic = {
            'densidad_aire_frente': densidad_aire_frente,
            'pt': pt,
            'ps': ps,
            'velocidad_sensor': velocidad_sensor
        }
        titulo = "Resultados"
        mostrar_en_tabla_dinamica(titulo, values_dic)
        mostrar_formula(" velocidad_sensor = sqrt(2*(pt-ps)/densidad_aire_frente)")
        # print(f"pt2: {pt2}, ps2: {ps2},densidad_aire_frente: {densidad_aire_frente}, velocidad_sensor 2: {velocidad_sensor}")
        mostrar_resultado(velocidad_sensor)
        return velocidad_sensor

    def calculate_Q2(self):
        """
            
            Crear variable Q2) Caudal sensor 2 (Q2) m3/s = velocidad sensor 2 (m/s)*Área ducto (m2)
        Returns:
            _type_: _description_
        """
        if self.Q2 != None:
            return self.Q2

        # calcular densidad aire en la frente
        HRf  = self.sensorData["q2"].mean()
        # temperatura bulbo seco
        Tbs2 = self.sensorData["lc"].mean()
        # presion barometrica en la frente
        P2  = self.sensorData["densidad2"].mean()
        # definir variables
        pt2 = self.sensorData["pt2"].mean()

        ps2 = self.sensorData["ps2"].mean()
        
        mostrar_inicio_formula("Calculando Q2", "")
        velocidad_sensor_2 = self.calcular_velocidad_sensor(Tbs2, HRf, P2, pt2, ps2, "solicitado desde Q2" )
        area_ducto = self.calcular_area_ducto()
        # = J22  * E24
        Q2 = velocidad_sensor_2 * area_ducto  #  caudal sensor 2 = (m/s)/(m2)
        
        values_dic = {
            'velocidad_sensor_2': velocidad_sensor_2,
            'pt2': pt2,
            'ps2': ps2,
            'area_ducto': area_ducto
        }

        mostrar_en_tabla_dinamica("valores del sistema y calculados para Q2",values_dic)
        mostrar_formula("Q2 = velocidad_sensor_2 * area_ducto")
        mostrar_resultado(Q2)
        # asignar al entorno global
        self.Q2 = Q2
        return Q2

    def calculate_Q1(self):
        #definir las variables de los sensores y el proyecto
        # ---> sensores
        # definir variables
        
        # comprobar si ya fue calculada
        if self.Q1 != None:
            return self.Q1
        
        mostrar_inicio_formula("Calculando el valor de Q1")
        pt1 = self.sensorData["pt1"].mean()
        ps1 = self.sensorData["ps1"].mean()
        tbs = self.sensorData["tbs"].mean() # humedad relativa
        hr = self.sensorData["hr"].mean() # temperatura bulbo seco
        P1 = self.sensorData["densidad1"].mean()  #  Presión barométrica ventilador
        tbs1 = self.sensorData["lc"].mean() # 
        velocidad_sensor_1 = self.calcular_velocidad_sensor(tbs, hr, P1, pt1, ps1, "solicitado desde Q1" )
        # velocidad_sensor_1 = self.calcular_velocidad_sensor1()   # 2 decimales

        area_ducto = self.calcular_area_ducto()
        Q1  = velocidad_sensor_1 *area_ducto #  m3/s = (m2)*(m/s).  (Crear variable Q1) caudal_ventilador_2
        values_dic = {
            'velocidad_sensor_1': velocidad_sensor_1,
            'pt2': pt1,
            'ps1': ps1,
            'area_ducto': area_ducto
        }

        mostrar_en_tabla_dinamica("valores del sistema y calculados para Q1", values_dic)
        mostrar_formula("Q1  = velocidad_sensor_1 *area_ducto")
        mostrar_resultado(Q1)
        separador()
        # asignar a las variables del entorno global 
        self.Q1 = Q1  # guardar el resultado en el entorno global para usarlo en otros metodos
        return Q1

    def calcular_semaforo_v1(self, Qf):
        equipamiento_diesel = self.project.equipamientos.all()
        
        caudal_requerido = 0 
        for equipo in equipamiento_diesel:
            caudal_requerido += equipo.qr_calculado
        dic_ = {
            "Qf": Qf,
            "caudal requerido": caudal_requerido
        }
        mostrar_en_tabla_dinamica("Variables utilizadas", dic_)
        if caudal_requerido < Qf:
            mostrar_semaforo("rojo")
            return "rojo"
        if caudal_requerido > Qf :
            mostrar_semaforo("verde")
            return "verde"
        raise Exception("No se logro calcular un valor para el semaforo")
    

    def caudal_en_la_frente_v1(self):
        """
        Calcula el caudal en la frente del ventilador
        formula: Qf (caudal de la frente) m3/s = Qf = Q2 - Lc*0.5*pt2*Lf /(100000);

        Returns:
            float: resultado de la aplicacion de la formula
        """
        # definir las variables requeridas
        mostrar_inicio_formulas_principales("calculando caudal en la frente (V1)", "")
        Q2 = self.calculate_Q2()
        Lc = self.leakage_coefficient_v4()
        pt2 = self.sensorData["pt2"].mean()
        lf = self.project.lf

        Qf = Q2 - Lc*0.5*pt2*(lf/100000)
        mostrar_formula("Qf = Q2 - Lc*0.5*pt2*(lf/100000)")
        values_dic = {
            'Q2 calculada': Q2,
            'Lc calculada': Lc,
            'pt2 ': pt2,
            'lf': lf
        }
        caudal_de_frente = {
        'estado': 1,
        'q_requerido': 0.3,
        'q_frente': 0.36,
    }
        mostrar_en_tabla_dinamica("Valores para calcular el caudal en la frente v1 ", values_dic)
        mostrar_resultado( Qf)
        self.calcular_semaforo_v1(Qf)

        separador()

        return Qf
    
    def calculate_tbh(self, tbs, hr):
        return tbs * atan(0.151977 * sqrt(hr + 8.313659)) + atan(tbs + hr) - atan(hr - 1.6763) + 0.00391838 * pow(hr, 1.5) * atan(0.023101 * hr) - 4.686035

    def calcular_semaforo_v2(self, velocidad_del_aire):
        if velocidad_del_aire < 0.25 and velocidad_del_aire < 2.5:
            mostrar_formula(f"0.25 < {velocidad_del_aire} < 2.5")
            mostrar_semaforo("verde")
            return "verde"
        mostrar_semaforo("verde")
        mostrar_formula(f"0.25 < velocidad_del_aire < {velocidad_del_aire}")
        return "rojo"



    def velocidad_del_aire_v2(self):
        """
        se encarga de calcular la densidad del aire en la frente del ventilador

        formula: Densidad aire en la frente = (P2-e)/(287.04*(Tbs2+273.15)); // Kg aire seco/m3

        Returns:
            float: densidad del aire en la frente
        """
        #definir las variables de los sensores y el proyecto
        # ---> sensores

        caudal_ventilador = self.caudal_en_la_frente_v1()
        mostrar_inicio_formulas_principales("Calculando velocidad del aire v2", "formula: velocidad_del_aire = Q1/Area_galeria")
        Q1 = self.calculate_Q1()
        Area_galeria = self.project.area_galeria 
        velocidad_del_aire = Q1/Area_galeria
        dict_result  = {
            'Q1': Q1,
            'Area_galeria': Area_galeria
        }
        mostrar_en_tabla_dinamica("Datos utilizados para la velocidad del aire" ,dict_result)
        mostrar_resultado(velocidad_del_aire)
        self.calcular_semaforo_v2(velocidad_del_aire)
        separador()
        return velocidad_del_aire
    
    def calcular_estado_v3(self, tgbh):
        # Crear el DataFrame
        data = {
            "trabajo continuo": [30, 26.7, 25],
            "75-25": [30.6, 28, 25.9],
            "50-50": [31.4, 29.4, 27.9],
            "25-75%": [32.2, 31.1, 30]
        }

        # Definir los índices
        indices = ["liviana", "moderada", "pesada"]

        # Crear el DataFrame con el índice especificado
        df = pd.DataFrame(data, index=indices)
        nivel_carga = self.project.nivel_carga
        print(df.loc[nivel_carga])
        fila = df.loc[nivel_carga]
        minimo = fila.iloc[0]
        maximo = fila.iloc[-1]
        if tgbh < minimo :
            mostrar_semaforo("rojo")
            return "rojo"
        
        if tgbh > maximo:
            mostrar_semaforo("rojo")
            return "rojo"
        
        mostrar_semaforo("verde")
        return "verde"
    
    def tgbh_v3(self):
        """
            TGBH = 0.7*t° bulbo humedo + 0.3*t°bulbo seco
            tbh : temperatura bulbo humedo 
            tbs : temperatura bulbo seco
        Returns:
            TGBH (float): resultado de la formulat TBGH
        """
        mostrar_inicio_formulas_principales("Calculando tgbh v3",f'''
                tgbh = (0.7 * {self.sensorData["tbh"].mean()}) + (0.3 * {self.sensorData["tbs"].mean()}, <br/>
                tgbh = (0.7 * self.sensorData["tbh"].mean()) + (0.3 * self.sensorData["tbs"].mean()
        ''')
        tgbh = (0.7 * self.sensorData["tbh"].mean()) + (0.3 * self.sensorData["tbs"].mean())
        self.calcular_estado_v3(tgbh)
        mostrar_resultado(tgbh)
        separador()
        return tgbh
    
    def calcular_semaforo_v4(self, lc):
        if lc < 0.5:
            mostrar_semaforo("verde")
            return "verde"
        
        if lc > 0.5 and lc < 1:
            mostrar_semaforo("amarillo")
            return "amarillo"
        
        if lc > 1:
            mostrar_semaforo("rojo")
            return "rojo"
        return 0
    

    def leakage_coefficient_v4(self):
        Q1 = self.calculate_Q1()
        Q2 = self.calculate_Q2()

        pt1 = self.sensorData["pt1"].mean() 
        pt2 = self.sensorData["pt2"].mean()
         
        L = self.project.dis_e_sens
        mostrar_inicio_formula("Calculando el Leakage Coefficient V4")

        Lc = 3*(Q1-Q2)*(pt1-pt2)/(2*L*(pow(pt1,1.5)-pow(pt2,1.5)))*100*pow(1000,0.5)
        # print(f"Variables de lc: Q1: {Q1}, Q2: {Q2}, pt1: {pt1}, pt2: {pt2}, L: {L}, Lc: {Lc}")
        # print(f"V4: leakage ({Lc})")

        values_dic = {
            'Q1': Q1,
            'Q2': Q2,
            'pt1': pt1,
            'pt2': pt2,
            'L': L,
            'Lc': Lc
        }
        mostrar_formula("Lc = 3*(Q1-Q2)*(pt1-pt2)/(2*L*(pow(pt1,1.5)-pow(pt2,1.5)))*100*pow(1000,0.5)")
        mostrar_en_tabla_dinamica("Valores utilizados, del sistema y calculados", values_dic)
        mostrar_resultado(Lc)
        separador()
        return Lc 

    def calcular_semaforo_v5(self):
        return 0 
    

    def punto_de_stall_v5(self):
        mostrar_inicio_formulas_principales("Calculando punto de stall V5", "")
        pt2 = self.sensorData["pt2"].mean()
        
        presion_total_df =  presion_total(self.project, self.vdfData, self.sensorData)

        # obtener el valor maximo del dataframe que contiene la curva ajustada
        presion_maxima_curvaAjustada = presion_total_df['presion'].max()
        fila = presion_total_df.loc[presion_total_df['presion'] == presion_maxima_curvaAjustada ]
    
   
        print(f"0: {fila['caudal']} 1: {fila['presion']}")
        print(presion_total_df)
        values = {
            "pt2": pt2,
            "presion maxima curvaAjustada": presion_maxima_curvaAjustada
        }
        mostrar_en_tabla_dinamica("Valores para la formula", values)
        
        stall = pt2 / presion_maxima_curvaAjustada
        mostrar_resultado(stall)
        separador()
        return stall

    def fugas_v6(self):
        mostrar_inicio_formulas_principales("Calculando las fugas V6","")
        # objetivo 30 minutos 
        media_hora_seg = 30*60

        presion_total_ventilador = SensorsData.objects.all().last() 
        fecha_hora = presion_total_ventilador.ts 

        anterior = SensorsData.objects.exclude(id=presion_total_ventilador.id).last()
        fecha_hora_anterior = anterior.ts

        diferencia = fecha_hora - fecha_hora_anterior

        segundos_diferencia = diferencia.total_seconds()

        
        if segundos_diferencia > 0:
            unidades_faltantes = (media_hora_seg / segundos_diferencia)
            unidades_faltantes = max(0, unidades_faltantes)
        else:
            unidades_faltantes = float("inf")
        
        queryset = SensorsData.objects.all().order_by('-id')
        print(f"{unidades_faltantes} registros para un total de 30 minutos")
        ultimos_30mins = SensorsData.objects.all().order_by('-id')[:int(unidades_faltantes)+3]
        primero = ultimos_30mins[0]
        ultimo = ultimos_30mins[len(ultimos_30mins) - 1]

        presion_actual = ultimo.pt1 
        presion_hace30m = primero.pt1
        dict_data = {
            "intervalo en segundos": segundos_diferencia,
            "presion actual": presion_actual,
            "presion hace30m": presion_hace30m
        }
        mostrar_en_tabla_dinamica("datos utilizados" ,dict_data)
        porcentaje = 1 - (presion_hace30m/presion_actual)

        mostrar_resultado(porcentaje)
        return 0
    
    def potencia_v7(self):
        mostrar_inicio_formulas_principales("calculando la potencia V7","")
        power = self.project.ventilador.hp
        vdf_data = VdfData.objects.all().last()
        potencia_consumida = vdf_data.power 
        dict_data  ={
            "power": power,
            "potencia consumida": potencia_consumida
        }
        mostrar_en_tabla_dinamica("datos utilizados" ,dict_data)
        potencia = (potencia_consumida/power)*100
        mostrar_resultado(potencia)
        separador()
        return potencia
    
    def calcular_estado_final(self, project):
        self.encender(project)
        #self.caudal_en_la_frente_v1()
        #self.velocidad_del_aire_v2()
       # self.tgbh_v3()
        #self.leakage_coefficient_v4()
        self.punto_de_stall_v5()
        #self.fugas_v6()
        #self.potencia_v7()
    

    def calculate_k(self):

        mostrar_inicio_formulas_principales("Calculando el valor de K","K (factor de fricción ducto) kg/m3 = (ps1-ps2)*(pow(Área ducto,3))/(Q1*Q2*Perímetro ducto*L)")
        ps1 = self.sensorData['ps1'].mean()
        ps2 = self.sensorData['ps2'].mean()
        area_ducto = self.calcular_area_ducto()
        Q1 = self.calculate_Q1()
        Q2 = self.calculate_Q2()
        #perimetro_ducto = self.

        separador()
        return 0


#### configurar jupyter para utilizar el framework django


In [ ]:
from asgiref.sync import sync_to_async
from applications.getdata.models import Proyecto

@sync_to_async
def get_last_project():
    return Proyecto.objects.all().order_by('-id').last() 
    
@sync_to_async
def ejecutar_semaforo(project, semaforo):
    semaforo.calcular_estado_final(project)

# Crear un semáforo
project = await get_last_project()
semaforo = Semaforo()
await ejecutar_semaforo(project, semaforo)



In [8]:

def decirle_a_paola_que_la_amo():
    print("Paolita te Amo mi amor!!")


decirle_a_paola_que_la_amo()





# TAREA 1
# tarea 1: aca debe practicar hacer una funcion que permita escribir un mensaje

def imprimirMensaje():
    print("Te amo mi ingeniero precioso")

# TAREA 2
# tarea 2: Escribir una funcion que permita sumar dos numeros y debe imprimir el resultado






Paolita te Amo mi amor!!
